In [3]:
import csv
import json
import pandas as pd
import os
import networkx as nx
import obonet as obo
import requests as req
import collections
import gzip
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import IPython, urllib.parse
#import urllib.requests
import requests as req
import biomedkg_utils
from biomedkg_utils import *
%matplotlib inline

## Download files
Adapted from https://github.com/dhimmel/bgee/blob/master/download/download.sh

In [34]:
urls = ['https://bgee.org/ftp/bgee_v13_2/download/calls/diff_expr_calls/Homo_sapiens_diffexpr-anatomy-simple.tsv.zip']

for url in urls:
    filename = url.split('/')[-1] 
    #os.remove(filename[:-4])
    !wget -N -P input/ $url        # Download file
    !unzip input/$filename        # Unzip file
    #!mv filename[:-4] input/$filename[:-4] 

--2022-07-15 18:34:56--  https://bgee.org/ftp/bgee_v13_2/download/calls/diff_expr_calls/Homo_sapiens_diffexpr-anatomy-simple.tsv.zip
Resolving bgee.org (bgee.org)... 130.223.50.57
Connecting to bgee.org (bgee.org)|130.223.50.57|:443... connected.
HTTP request sent, awaiting response... 200 OK
Server ignored If-Modified-Since header for file ‘input/Homo_sapiens_diffexpr-anatomy-simple.tsv.zip’.
You might want to add --no-if-modified-since option.

Archive:  input/Homo_sapiens_diffexpr-anatomy-simple.tsv.zip
  inflating: Homo_sapiens_diffexpr-anatomy-simple.tsv  


### Load entrez gene for ensembl conversion

In [2]:
# Get Entrez Gene IDs
temp = pd.read_csv('output/nodes/genes_nodes.csv')[['Gene (Entrez)']]
entrez_genes = sorted([gene.split(':')[1] for gene in list(temp['Gene (Entrez)'])[1:]])
#entrez_genes = sorted(list(json.load(open('output/protein2gene/all_entrez2uniprot.json')).keys()))[1:]
entrez_ids = str(",".join(entrez_genes))

In [3]:
# Use MyGene.Info to get Ensembl IDs
headers = {'content-type': 'application/x-www-form-urlencoded'}
params = 'q=%s&scopes=entrezgene&fields=ensembl&species=human'%entrez_ids
res = req.post('http://mygene.info/v3/query', \
                    data=params, \
                    headers=headers).json()

In [4]:
# Align/map Ensembl to Entrez
ensembl_is_entrez = dict()

for r in res:
    
    # Entrez ID
    try:entrez_id = r['_id']
    except:continue
    
    # Ensembl ID
    try:
        if type(r['ensembl']) == list:
            for entry in r['ensembl']:
                ensembl = entry['gene']
                ensembl_is_entrez[ensembl] = entrez_id # 1 to 1
                
        elif type(r['ensembl']) == dict:
            ensembl = r['ensembl']['gene']
            ensembl_is_entrez[ensembl] = entrez_id # 1 to 1
    except: continue

In [5]:
json.dump(ensembl_is_entrez, open('output/gene2gene/ensembl2entrez.json','w'))

In [2]:
ensembl_is_entrez = json.load(open('output/gene2gene/ensembl2entrez.json'))

### Input differential expression data

In [3]:
# Combined differential expression dataframe
diffex_df = pd.read_table('input/Homo_sapiens_diffexpr-anatomy-simple.tsv.zip', compression='zip')
diffex_df = diffex_df[diffex_df['Call quality']=='high quality']

# Under expressed genes
underexp_df = diffex_df[diffex_df['Differential expression']=='under-expression']

# Over expressed genes
overexp_df = diffex_df[diffex_df['Differential expression']=='over-expression']

display(diffex_df['Differential expression'].value_counts())
diffex_df.tail()

over-expression     167758
under-expression    156032
Name: Differential expression, dtype: int64

,Gene ID,Gene name,Anatomical entity ID,Anatomical entity name,Developmental stage ID,Developmental stage name,Differential expression,Call quality
794747,ENSG00000273492,AP000230.1,UBERON:0001134,skeletal muscle tissue,UBERON:0000113,post-juvenile adult stage,under-expression,high quality
794749,ENSG00000273492,AP000230.1,UBERON:0001987,placenta,UBERON:0000113,post-juvenile adult stage,under-expression,high quality
794750,ENSG00000273492,AP000230.1,UBERON:0002037,cerebellum,UBERON:0000113,post-juvenile adult stage,over-expression,high quality
794751,ENSG00000273492,AP000230.1,UBERON:0002046,thyroid gland,UBERON:0000113,post-juvenile adult stage,over-expression,high quality
794752,ENSG00000273492,AP000230.1,UBERON:0002367,prostate gland,UBERON:0000113,post-juvenile adult stage,under-expression,high quality


In [1]:
!grep -r "uberon2mesh.json" *.ipynb

anatomy_to_anatomy.ipynb:    "with open('output/anatomy2anatomy/uberon2mesh.json','w') as fout:\n",
gene_to_anatomy.ipynb:    "uberon2mesh = json.load(open('output/anatomy2anatomy/uberon2mesh.json'))\n",


## Gene -over/underexpressed- Anatomy

In [4]:
# UBERON -is- MeSH Anatomy
uberon2mesh = json.load(open('output/anatomy2anatomy/uberon2mesh.json'))

# Gene -overexpressed in- Anatomy
underexp_gene2anat, underexp_anat2gene = dict(), dict()
underexp_gene2anat, underexp_anat2gene = gene_expressed_in_anatomy(underexp_df, underexp_gene2anat, underexp_anat2gene, uberon2mesh, ensembl_is_entrez)

# Gene -underexpressed in- Anatomy
overexp_gene2anat, overexp_anat2gene  = dict(), dict()
overexp_gene2anat1, overexp_anat2gene1 = gene_expressed_in_anatomy(overexp_df, overexp_gene2anat, overexp_anat2gene, uberon2mesh, ensembl_is_entrez)

print('Under', len(underexp_gene2anat), len(underexp_anat2gene))
print('Over', len(overexp_gene2anat1), len(overexp_anat2gene1))

# Prepare to export
underexp_gene2anat = switch_dictset_to_dictlist(underexp_gene2anat)
overexp_gene2anat = switch_dictset_to_dictlist(overexp_gene2anat)

# Export
json.dump(underexp_gene2anat, open('output/gene2anatomy/underexpressed_gene2anat.json','w'))
json.dump(overexp_gene2anat, open('output/gene2anatomy/overexpressed_gene2anat.json','w'))

16437 70
18914 70
Under 16437 70
Over 18914 70


## Export Edges

In [5]:
# Output underexpressed genes
output_edgefile_onerel_noweight(
    outpath = 'output/gene2anatomy/edges_gene-underexpressed_in-anatomy.csv', 
    columns = ['Gene (Entrez)', 'Anatomy (MeSH)','Relationship'], 
    dictionary = underexp_gene2anat, 
    rel = '-underexpressed_in->', 
    prefix_col1='Entrez:', 
    prefix_col2='MeSH_Anatomy:')

In [6]:
# Output overexpressed genes
output_edgefile_onerel_noweight(
    outpath = 'output/gene2anatomy/edges_gene-overexpressed_in-anatomy.csv', 
    columns = ['Gene (Entrez)', 'Anatomy (MeSH)','Relationship'], 
    dictionary = overexp_gene2anat, 
    rel = '-overexpressed_in->', 
    prefix_col1='Entrez:', 
    prefix_col2='MeSH_Anatomy:')